In [1]:
%matplotlib inline

In [ ]:
# Julie Baker
# VacationPy
# June 2021

# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import gkey 

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Study data files
weather_path = os.path.join(os.getcwd(), "cities.csv")

# convert weather data to df
weather_df = pd.read_csv(weather_path)

In [4]:
weather_df

,Unnamed: 0,City,Latitude,Longitude,Temperature (F),Maximum Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Time,Time Zone
0,0,dikson,73.5069,80.5464,32.00,32.00,84,0,5.03,RU,1624218521,25200
1,1,barstow,34.8986,-117.0228,104.47,106.79,8,0,26.53,US,1624218522,-25200
2,2,samoylovka,51.1840,43.7081,65.62,65.62,67,51,8.61,RU,1624218523,14400
3,3,airai,-8.9266,125.4092,55.53,55.53,92,34,3.27,TL,1624218525,32400
4,4,celestun,20.8667,-90.4000,91.08,91.08,59,92,6.44,MX,1624218527,-18000
...,...,...,...,...,...,...,...,...,...,...,...,...
538,538,bonavista,48.6499,-53.1147,60.80,60.80,98,100,15.01,CA,1624219015,-9000
539,539,ranong,9.9658,98.6348,79.56,79.56,87,98,8.72,TH,1624219015,25200
540,540,marsh harbour,26.5412,-77.0636,83.57,83.57,75,64,16.60,BS,1624219016,-14400
541,541,copiapo,-27.3667,-70.3333,56.89,56.89,44,25,6.42,CL,1624219016,-14400


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key=gkey)

# Store latitude and longitude in locations
locations = weather_df[["Latitude", "Longitude"]]
# Store humidity
humidity = weather_df["Humidity (%)"].astype(float)
# coordinates for center
map_center = (0, 0)
# choosing map_type
which_map = 'HYBRID'

In [10]:
# Plot Heatmap
fig = gmaps.figure(map_type=which_map, center=map_center, zoom_level=1)
#fig = gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

In [7]:
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
# ideal weather conditions

In [27]:
# renaming columns to be shorter
weather_df = weather_df.rename(columns={'Latitude':'Lat','Longitude':'Lng', 'Temperature (F)': 'Temp', 'Maximum Temperature (F)': 'Max', 'Humidity (%)': 'Hum', 'Cloudiness (%)': 'Cld', 'Wind Speed (mph)': 'Wind', 'Country': 'Country', 'Time Zone': 'TZ'})
weather_df.head() # cities = 543

,Unnamed: 0,City,Lat,Lng,Temp,Max,Hum,Cld,Wind,Country,Time,TZ
0,0,dikson,73.5069,80.5464,32.00,32.00,84,0,5.03,RU,1624218521,25200
1,1,barstow,34.8986,-117.0228,104.47,106.79,8,0,26.53,US,1624218522,-25200
2,2,samoylovka,51.1840,43.7081,65.62,65.62,67,51,8.61,RU,1624218523,14400
3,3,airai,-8.9266,125.4092,55.53,55.53,92,34,3.27,TL,1624218525,32400
4,4,celestun,20.8667,-90.4000,91.08,91.08,59,92,6.44,MX,1624218527,-18000


In [12]:
# temperature max temp 65 to 85
not_too_cool_df = weather_df.loc[weather_df['Max'] > 65, :] # cities = 343
not_too_hot_df = not_too_cool_df.loc[weather_df['Max'] < 85, :] # cities = 268

In [13]:
# humidity 30 to 70%
not_too_humid_df = not_too_hot_df.loc[weather_df['Hum'] < 70, :] # cities = 95
not_too_dry_df = not_too_humid_df.loc[weather_df['Hum'] > 30, :] # cities = 82

In [14]:
# wind speed less than 10
not_windy_df = not_too_dry_df.loc[weather_df['Wind'] <10, :] # cities = 53
lil_breeze_df = not_windy_df.loc[weather_df['Wind'] >2.5, :] # cities = 47

In [15]:
# cloudiness less than 20%
ok_clouds_df = lil_breeze_df.loc[weather_df['Cld'] <20, :] # cities = 15

In [16]:
just_right_df = ok_clouds_df.dropna() # cities = 13

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
pd.options.mode.chained_assignment = None # was sometimes getting an error with this, so found this solution online
hotel_df = just_right_df
hotel_df['Hotel Name'] = ''

In [18]:
hotel_df

,Unnamed: 0,City,Lat,Lng,Temp,Max,Hum,Cld,Wind,Country,Time,TZ,Hotel Name
69,69,arraial do cabo,-22.9661,-42.0278,80.46,84.16,61,0,5.75,BR,1624218467,-10800,
84,84,kruisfontein,-34.0033,24.7314,69.89,69.89,31,8,8.55,ZA,1624218632,7200,
171,171,nikolskoye,59.7035,30.7861,74.34,76.89,68,0,3.44,RU,1624218454,10800,
189,189,plettenberg bay,-34.0527,23.3716,74.19,74.19,36,8,7.16,ZA,1624218750,7200,
190,190,east london,-33.0153,27.9116,70.54,70.54,35,5,7.20,ZA,1624218462,7200,
208,208,makakilo city,21.3469,-158.0858,78.55,80.62,61,1,6.91,US,1624218768,-36000,
234,234,colorado,-22.8375,-51.9731,66.29,66.29,61,7,6.89,BR,1624218791,-10800,
256,256,westlock,54.1502,-113.8688,62.64,67.41,55,5,5.75,CA,1624218809,-21600,
266,266,saskylakh,71.9167,114.0833,75.43,75.43,40,13,5.57,RU,1624218819,32400,
337,337,zhangye,38.9342,100.4517,67.82,67.82,35,3,7.00,CN,1624218893,28800,


In [20]:
# find the closest hotel for each set of coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    # "rankby": "distance",  # this prevented results from coming back, so pulled it out
    "types": "lodging",
    "key": gkey
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get lat & lng from df
    lat = row['Lat']
    lng = row['Lng']
    lat_lng = f"{lat}, {lng}"
    city = row['City']
    country = row['Country']

    # add keyword to params dict
    params['location'] = f"{lat}, {lng}"
 
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()

    # extract results
    results = response['results']
  
    try:
        print(f"Closest lodging in {city}, {country} is {results[0]['name']}.")
        print(f"Hotel is located at {results[0]['vicinity']}.")
        print(f"Hotel rating is {results[0]['rating']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Address'] = results[0]['vicinity']
        # hotel_df.loc[index, 'price_level'] = results[0]['price_level']
        hotel_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")   
    print("------------")

Retrieving Results for Index 69: arraial do cabo.
Closest lodging in arraial do cabo, BR is Pousada Porto Praia.
Hotel is located at Rua Vilas Bôas, 161 - Macedônia, Arraial do Cabo.
Hotel rating is 4.3.
------------
Retrieving Results for Index 84: kruisfontein.
Closest lodging in kruisfontein, ZA is Oyster Bay House Rental.
Hotel is located at 351 Kabeljou street, Oysterbay, Humansdorp.
Hotel rating is 3.7.
------------
Retrieving Results for Index 171: nikolskoye.
Closest lodging in nikolskoye, RU is Tourist House - Sablino.
Hotel is located at Parkovaya Ulitsa, 26/1, Ul'yanovka.
Hotel rating is 4.2.
------------
Retrieving Results for Index 189: plettenberg bay.
Closest lodging in plettenberg bay, ZA is Thanda Vista B&B.
Hotel is located at 8 Susan Street, Plettenberg Bay.
Hotel rating is 4.7.
------------
Retrieving Results for Index 190: east london.
Closest lodging in east london, ZA is Tu Casa.
Hotel is located at 2 Ascot Road, Nahoon, East London.
Hotel rating is 4.6.
--------

In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# Create symbol layer
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))